In [1]:
import earthaccess
import rioxarray
import rasterio
import xarray as xr
import numpy as np
from timebudget import timebudget

In [ ]:
results = earthaccess.search_data(
    short_name="MUR-JPL-L4-GLOB-v4.1",
    #temporal=("2020-01-01", "2021-12-31"),
    temporal=("2019-01-01", "2019-01-31"),
)


Granules found: 31


In [ ]:
@timebudget
def via_earthaccess():
  files = earthaccess.open(results)
  ds = xr.open_mfdataset(files,
                         decode_times=False, 
                         parallel=True, 
                         variables=['analysed_sst', 'sea_ice_fraction'], 
                         concat_dim="time", 
                         combine="nested")
  return(ds)

ds2 = via_earthaccess()


In [ ]:

dds = ds2.sel(lon=slice(-93, -76), lat=slice(41, 49))
cond = (dds.sea_ice_fraction < 0.15) | np.isnan(dds.sea_ice_fraction)
result = dds.analysed_sst.where(cond)
result.mean("time").plot(figsize=(14, 6), x="lon", y="lat")

In [ ]:

@timebudget
def via_gdalvsi():
    ## pull out the URLs
    data_links = [granule.data_links(access="external") for granule in results]
    url_links = [f'{link[0]}' for link in data_links]
    # and here we go
    ds = xr.open_mfdataset(url_links, engine = "rasterio", decode_times=False, parallel=True)
    return(ds)

ds1 = via_gdalvsi()

In [ ]:

import os
from pathlib import Path
cookies = os.path.expanduser("~/.urs_cookies")
Path(cookies).touch()

  data_links = [granule.data_links(access="external") for granule in results]
  url_links = [f'{link[0]}' for link in data_links]
  # and here we go
with rasterio.Env(GDAL_HTTP_COOKIEFILE=cookies, 
                    GDAL_HTTP_COOKIEJAR=cookies, 
                    GDAL_HTTP_NETRC=True):
    with WarpedVRT(url_links, src_crs='EPSG:4326') as vrt:


@timebudget
def via_gdalvsi():
  ## pull out the URLs

    ds = xr.open_mfdataset(url_links, engine = "rasterio", decode_times=False, parallel=True)
    return(ds)


ds1 = via_gdalvsi()

In [ ]:
dds = ds1.sel(x=slice(-93, -76), y=slice(41, 49)) # crs is messed up
dds.values()

In [ ]:
dds = ds1.sel(x=slice(-93, -76), y=slice(41, 49))
cond = (dds.sea_ice_fraction < 0.15) | np.isnan(dds.sea_ice_fraction)
result = dds.analysed_sst.where(cond)
result.std("time").plot(figsize=(14, 6), x="x", y="y")